<a href="https://colab.research.google.com/github/SnipeHunting/MQ135-Air-Quality-with-ST7735-Display/blob/master/Options.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://plainenglish.io/blog/a-simple-guide-to-plotly-for-plotting-financial-chart-54986c996682
#https://medium.com/@txlian13/webscrapping-options-data-with-python-and-yfinance-e4deb0124613

#import sys
#!{sys.executable} -m pip install yfinance --upgrade --no-cache-dir



#import sys
#!{sys.executable} -m pip install yoptions

#import sys
#!{sys.executable} -m pip install requests-cache

#import sys
#!{sys.executable} -m pip install datetime


#!{sys.executable} -m pip install matplotlib

import pandas as pd
import numpy as np
import yoptions as yo
import yfinance as yf
import requests_cache
import datetime
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt


In [ ]:
ticker = "AMAT"

In [ ]:
tk = yf.Ticker(ticker)
tk.dividends

Date
2005-05-16 00:00:00-04:00    0.03
2005-08-15 00:00:00-04:00    0.03
2005-11-15 00:00:00-05:00    0.03
2006-02-14 00:00:00-05:00    0.03
2006-05-16 00:00:00-04:00    0.05
                             ... 
2021-11-24 00:00:00-05:00    0.24
2022-02-22 00:00:00-05:00    0.24
2022-05-25 00:00:00-04:00    0.26
2022-08-24 00:00:00-04:00    0.26
2022-11-23 00:00:00-05:00    0.26
Name: Dividends, Length: 71, dtype: float64

In [ ]:
fig = go.Figure()

def options_chain(symbol):

    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options
    nearMonths = 5 # next 5 expirations and ignore far options
    callChar = 'C'
  
    # Get options for each expiration
    options = pd.DataFrame()

    for e in exps[0:nearMonths]:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls)
        opt['expirationDate'] = e

        fig.add_trace(
          go.Scatter(x=opt.strike,y=opt.impliedVolatility, 
                    mode='lines+markers', 
                    name='Call '+str(e)))
 
        options = options.append(opt, ignore_index=True)

    for e in exps[0:nearMonths]:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.puts)
        opt['expirationDate'] = e

        fig.add_trace(
          go.Scatter(x=opt.strike,y=opt.impliedVolatility, 
                    mode='markers', 
                    name='Puts '+str(e)))
 
        options = options.append(opt, ignore_index=True)

    
    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    #options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    #options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary and meaningless columns
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

#    for e in exps[::nearMonths]:



    fig.update_layout(
        title = 'Implied Volatility - '+ticker,
        xaxis_title= 'Strike',
        yaxis_title = 'IV'
    )


    fig.show()
    return options

In [ ]:
options_chain(ticker)


,contractSymbol,strike,bid,ask,volume,openInterest,impliedVolatility,inTheMoney,expirationDate,CALL,mark
0,AMAT230203C00065000,65.0,47.30,48.40,NaN,2,2.578129,True,2023-02-03,True,47.850
1,AMAT230203C00080000,80.0,32.25,33.65,1.0,2,1.805665,True,2023-02-03,True,32.950
2,AMAT230203C00083000,83.0,29.30,30.55,NaN,3,1.623049,True,2023-02-03,True,29.925
3,AMAT230203C00084000,84.0,28.30,29.45,NaN,1,1.530276,True,2023-02-03,True,28.875
4,AMAT230203C00085000,85.0,27.30,28.60,5.0,3,1.537112,True,2023-02-03,True,27.950
...,...,...,...,...,...,...,...,...,...,...,...
394,AMAT230303P00112000,112.0,4.80,5.20,16.0,9,0.404791,False,2023-03-03,False,5.000
395,AMAT230303P00113000,113.0,5.25,5.65,NaN,4,0.399298,True,2023-03-03,False,5.450
396,AMAT230303P00114000,114.0,5.75,6.10,2.0,51,0.391608,True,2023-03-03,False,5.925
397,AMAT230303P00118000,118.0,8.10,8.55,NaN,1,0.387824,True,2023-03-03,False,8.325


In [ ]:
TRADING_DAYS = 252

# Pull NIFTY data from Yahoo finance
stockData = yf.download(ticker,start='2018-6-1', end='2023-1-29')

returns = np.log(stockData['Close']/stockData['Close'].shift(1))
returns.fillna(0, inplace=True)
volatility = returns.rolling(window=TRADING_DAYS).std()*np.sqrt(TRADING_DAYS)*100
volatility.tail()

[*********************100%***********************]  1 of 1 completed


Date
2023-01-23 00:00:00-05:00    49.430698
2023-01-24 00:00:00-05:00    49.353992
2023-01-25 00:00:00-05:00    49.261027
2023-01-26 00:00:00-05:00    49.071261
2023-01-27 00:00:00-05:00    49.101679
Name: Close, dtype: float64

In [ ]:
from tables import index
fig3 = go.Figure()

fig3 = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.01,
                    row_heights=[10,10,10])

fig3.add_trace(go.Candlestick(x=stockData.index,
                             open=stockData['Open'],
                             high=stockData['High'],
                             low=stockData['Low'],
                             close=stockData['Close'],
                             name = ticker
                             ),
               row=1, 
               col=1
               )


# removing all empty dates
# build complete timeline from start date to end date
dt_all = pd.date_range(start=stockData.index[0],end=stockData.index[-1])
# retrieve the dates that ARE in the original datset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(stockData.index)]
# define dates with missing values
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])


# hide dates with no values
fig3.update_xaxes(rangebreaks=[dict(values=dt_breaks)])
# remove rangeslider
fig3.update_layout(xaxis_rangeslider_visible=False)
# add chart title
fig3.update_layout(title=ticker)


# Plot volatility




fig3.add_trace(go.Scatter(x=stockData.index,
                     y=volatility, name = 'HV'
                    ), row=2, col=1)






fig3.add_trace(go.Scatter(x=stockData.index,
                     y=tk.dividends, name = 'Dividends'
                    ), row=3, col=1)


fig3.show()

In [ ]:
sd = stockData
div = tk.dividends
#div2 = pd.DataFrame(sd)

div2['New Columb'] = 0



#options['New Columb'] = options['contractSymbol'].str[4:].apply(
#    lambda x: "C" in x)

#div = stockData
#div2 = pd.DataFrame(sd)

#new = div2.merge(div,how='inner',on=['Date'])
div





Date
2005-05-16 00:00:00-04:00    0.03
2005-08-15 00:00:00-04:00    0.03
2005-11-15 00:00:00-05:00    0.03
2006-02-14 00:00:00-05:00    0.03
2006-05-16 00:00:00-04:00    0.05
                             ... 
2021-11-24 00:00:00-05:00    0.24
2022-02-22 00:00:00-05:00    0.24
2022-05-25 00:00:00-04:00    0.26
2022-08-24 00:00:00-04:00    0.26
2022-11-23 00:00:00-05:00    0.26
Name: Dividends, Length: 71, dtype: float64